In [ ]:
vxst = local_csv('vxstcurrent.csv',
                date_column="Date", 
                 use_date_column_as_index=True,
                 skiprows=[0,1,2],
                )

vxst = vxst.dropna()
vxst = vxst.rename(columns=dict(map(lambda x: (x, "vxst_"+x), vxst.columns)))

vxx = get_pricing("vxx", start_date=vxst.index[0], end_date=vxst.index[-1], )

In [ ]:
df = get_pricing(["AAPL", "FB"], fields="close_price")
df.columns = map(lambda x: x.symbol, df.columns)

In [ ]:
import talib as ta
import functools

myroc = functools.partial(ta.WMA, timeperiod=10)

df.apply(myroc, axis=0)

In [ ]:
import pandas as pd 
df = pd.concat([vxx, vxst],  axis=1)

In [ ]:
df

In [ ]:
df["vsxt_intraday"] = df["vxst_Open"]/df["vxst_Close"]-1
df["vxx_intraday"] = df["open_price"]/df["close_price"]-1
df["vsxt_intraday_prevday"] = df["vsxt_intraday"].shift(1)
df["vxst_highlow"] = (df["vxst_High"]-df["vxst_Low"])/df["vxst_Low"]
df["vxst_highlow_prevday"] = df["vxst_highlow"].shift(-1)
df["marketcap"] = df["volume"] * df["price"]
df["juriandate"] = df.index.to_julian_date()
df["vxx_dayreturn"] = df["price"].pct_change()

In [ ]:
df.plot.scatter(x="vxst_highlow_prevday", y="vxx_dayreturn", c="juriandate" )

In [ ]:
# Import the history function
from quantopian.research.experimental import history, continuous_future
cl = continuous_future('CL', adjustment=None)
cl_pricing = history(
    cl, 
    fields='price', 
    frequency='daily', 
    start='2015-10-21', 
    end='2016-06-01'
)



In [ ]:
cl_contracts = symbols(['CLF16', 'CLG16', 'CLH16', 'CLJ16', 'CLK16', 'CLM16'])

# Pricing data for our consecutive contracts from earlier.
cl_consecutive_contract_pricing = history(
    cl_contracts, 
    fields='price', 
    frequency='daily', 
    start='2015-10-21', 
    end='2016-06-01'
)

cl = continuous_future('CL', adjustment=None)
cl_pricing = history(
    cl, 
    fields='price', 
    frequency='daily', 
    start='2015-10-21', 
    end='2016-06-01'
)


cl_consecutive_contract_pricing.plot()
cl_pricing.plot(style='k--', legend="continuous")

In [ ]:
cl0 = continuous_future('CL', adjustment=None)
cl1 = continuous_future('CL', adjustment=None, offset=1)

cl_pricing = history(
    [cl0, cl1],
    fields='price', 
    frequency='daily', 
    start='2015-10-21', 
    end='2016-06-01'
)

cl_pricing.plot()

In [ ]:
from quantopian.research.experimental import continuous_future, history

cl_future = continuous_future('CL')
xb_future = continuous_future('XB')

start='2014-01-01'
end='2015-01-01'
    
cl_price = history(cl_future, start=start, end=end, fields='price', frequency='daily', )
xb_price = history(xb_future, start=start, end=end, fields='price', frequency='daily', )


In [ ]:
c = symbols("CLF16")
x = symbols("XBF16")

In [ ]:
c.multiplier, x.multiplier


In [ ]:
cl_price.plot()
xb_price.multiply(42).plot()

In [ ]:
import statsmodels.api as sm
import statsmodels.tsa as tsa

_, p, _ = tsa.stattools.coint(cl_price, xb_price)
p

In [ ]:
import pandas as pd
df = pd.DataFrame({"CL":cl_price, "XB":xb_price})


In [ ]:
df["CL_change"] = df["CL"].pct_change()
df["XB_change"] = df["XB"].pct_change()

In [ ]:
import scipy as sp 
term = 20 
df.plot(kind='scatter', x='CL_change',y='XB_change')